In [ ]:
# Austin Diamond

1. Import Pandas, Numpy, RegEx, and NLTK libraries

In [ ]:
#Import Libraries
import numpy as np
import re
import pickle
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import load_files
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

2. Import VADER Sentiment Analysis

In [ ]:
#Import Vader
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.downloader.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

3. Read the PhoneReviews excel data, only keep the Reviews column

In [ ]:
from google.colab import files
files.upload();

Saving PhoneReviews.xlsx to PhoneReviews (1).xlsx


In [ ]:
phone = pd.DataFrame(pd.read_excel("PhoneReviews.xlsx"))

In [ ]:
phone.head()

NameError: ignored

In [ ]:
#Drop all columns except Reviews
phonerev = phone[['Reviews']]

In [ ]:
#Make sure only Reviews is left
phonerev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Reviews  20 non-null     object
dtypes: object(1)
memory usage: 288.0+ bytes


4. Use lambda function to apply Vader sentiment to the text ot calculate Sentiment Intensity of all reviews, add the new column to the dataframe as scores

In [ ]:
analyzer = SentimentIntensityAnalyzer()

phonerev['neg'] = phonerev['Reviews'].apply(lambda x:analyzer.polarity_scores(x)['neg'])
phonerev['neu'] = phonerev['Reviews'].apply(lambda x:analyzer.polarity_scores(x)['neu'])
phonerev['pos'] = phonerev['Reviews'].apply(lambda x:analyzer.polarity_scores(x)['pos'])
phonerev['compound'] = phonerev['Reviews'].apply(lambda x:analyzer.polarity_scores(x)['compound'])

In [ ]:
phonerev

,Reviews,neg,neu,pos,compound
0,I feel so LUCKY to have found this used (phone...,0.015,0.796,0.189,0.8783
1,"nice phone, nice up grade from my pantach revu...",0.072,0.597,0.331,0.9231
2,Very pleased,0.000,0.238,0.762,0.4927
3,It works good but it goes slow sometimes but i...,0.000,0.500,0.500,0.9185
4,Great phone to replace my lost phone. The only...,0.038,0.897,0.065,0.2942
5,I already had a phone with problems... I know ...,0.143,0.857,0.000,-0.9107
6,The charging port was loose. I got that solder...,0.057,0.891,0.052,-0.0516
7,"Phone looks good but wouldn't stay charged, ha...",0.087,0.709,0.204,0.4486
8,I originally was using the Samsung S2 Galaxy f...,0.023,0.848,0.129,0.9491
9,It's battery life is great. It's very responsi...,0.000,0.791,0.209,0.8268


5. Add sentiment type by checking the coupund values (>0 Positive, <0 Negative, ==0 Neutral)

In [ ]:
phonerev['sentiment'] = pd.cut(x=phonerev['compound'], bins=[-1, -0.000001, 0, 1], labels=['neg','neu','pos'])

In [ ]:
phonerev

,Reviews,neg,neu,pos,compound,sentiment
0,I feel so LUCKY to have found this used (phone...,0.015,0.796,0.189,0.8783,pos
1,"nice phone, nice up grade from my pantach revu...",0.072,0.597,0.331,0.9231,pos
2,Very pleased,0.000,0.238,0.762,0.4927,pos
3,It works good but it goes slow sometimes but i...,0.000,0.500,0.500,0.9185,pos
4,Great phone to replace my lost phone. The only...,0.038,0.897,0.065,0.2942,pos
5,I already had a phone with problems... I know ...,0.143,0.857,0.000,-0.9107,neg
6,The charging port was loose. I got that solder...,0.057,0.891,0.052,-0.0516,neg
7,"Phone looks good but wouldn't stay charged, ha...",0.087,0.709,0.204,0.4486,pos
8,I originally was using the Samsung S2 Galaxy f...,0.023,0.848,0.129,0.9491,pos
9,It's battery life is great. It's very responsi...,0.000,0.791,0.209,0.8268,pos
